In [28]:
import pandas as pd

print 'vor einlesen'
genes = pd.read_csv('genes.csv',header=4, index_col=1)
print 'nach einlesen'
#print genes
sequence = genes[['SwissProt','GenBank','Name','Sequence']]
metabolite =pd.read_csv('metabolite.csv',header=4, index_col=1)
print metabolite.head()

vor einlesen
nach einlesen
                ID    BiGG    KEGG      CAS  PubChem   3DMET  ChEBI PDB-CCD  \
BioCyc                                                                        
CPD-3707    A23CMP  23camp  C02353      NaN     5400  B01564  27844     ACK   
CPD-3706      A3MP    3amp  C01367  84-21-9     4566  B01442  28931     3AM   
ACET            AC      ac  C00033  64-19-7     3335  B00009  15366     ACY   
ACETALD       ACAL   acald  C00084  75-07-0     3384  B01155  15343     ACE   
ACETYL-COA   ACCOA   accoa  C00024  72-89-9     2093  B04621  15351     ACO   

             KNApSAcK    LIPID MAPS   ...    Molecular Weight  \
BioCyc                                ...                       
CPD-3707          NaN           NaN   ...                 328   
CPD-3706    C00019350           NaN   ...                 345   
ACET        C00001176  LMFA01010002   ...                  59   
ACETALD     C00007392           NaN   ...                  44   
ACETYL-COA  C00007259        